In [148]:
import pandas as pd

data = pd.read_csv('train.txt', delimiter='\t')
del data['id']
data.loc[:,'label'] = data.loc[:,'label']
data

,turn1,turn2,turn3,label
0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,By,by Google Chrome,Where you live,others
3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,Just for time pass,wt do u do 4 a living then,Maybe,others
...,...,...,...,...
30155,I don't work,I could take your shift,I am a student,others
30156,I'm not getting you 😭😭😭,Why are you crying??,Because you are not making any sense,sad
30157,Haha,"no, seriously. What is up with that o-o",Had your breakfast?,others
30158,Do you sing?,yea a lil,Nice,others


In [149]:
sums = dict()
for label in data['label'].unique():
    sums[label] = sum(data['label']==label)
print(sums)

{'others': 14948, 'angry': 5506, 'sad': 5463, 'happy': 4243}


In [150]:
data['text'] = data.turn1+' '+data.turn2+' '+data.turn3
data

,turn1,turn2,turn3,label,text
0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others,Don't worry I'm girl hmm how do I know if you...
1,When did I?,saw many times i think -_-,No. I never saw you,angry,When did I? saw many times i think -_- No. I n...
2,By,by Google Chrome,Where you live,others,By by Google Chrome Where you live
3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry,U r ridiculous I might be ridiculous but I am ...
4,Just for time pass,wt do u do 4 a living then,Maybe,others,Just for time pass wt do u do 4 a living then ...
...,...,...,...,...,...
30155,I don't work,I could take your shift,I am a student,others,I don't work I could take your shift I am a st...
30156,I'm not getting you 😭😭😭,Why are you crying??,Because you are not making any sense,sad,I'm not getting you 😭😭😭 Why are you crying?? B...
30157,Haha,"no, seriously. What is up with that o-o",Had your breakfast?,others,"Haha no, seriously. What is up with that o-o H..."
30158,Do you sing?,yea a lil,Nice,others,Do you sing? yea a lil Nice


In [151]:
emotion_text = dict()
for label in data['label'].unique():
    emotion_text[label] = data[data['label']==label][['turn1', 'turn3']]

In [156]:
import emojis

import hunspell
from tokenizer import tokenizer

T = tokenizer.TweetTokenizer(regularize=False)
hobj = hunspell.HunSpell('/usr/share/hunspell/en_US.dic', '/usr/share/hunspell/en_US.aff')

exclamatives = dict.fromkeys(emotion_text.keys(),0)
interrogatives = dict.fromkeys(emotion_text.keys(),0)
upper_cases = dict.fromkeys(emotion_text.keys(),0)
lower_cases = dict.fromkeys(emotion_text.keys(),0)
emojis_s = dict.fromkeys(emotion_text.keys(),0)
unknown_words = dict.fromkeys(emotion_text.keys(),[])
unknown_words_len = dict.fromkeys(emotion_text.keys(),0)
avg_lengths = dict.fromkeys(emotion_text.keys(), 0)
for emotion, df in emotion_text.items():
    for sentence in df['turn1']+' '+df['turn3']:
        exclamatives[emotion] += sentence.count('?')
        interrogatives[emotion] += sentence.count('!')
        upper_cases[emotion] += sum(1 for l in sentence if l.isupper())
        lower_cases[emotion] += sum(1 for l in sentence if l.islower())
        emojis_s[emotion] += emojis.count(sentence)
        tokens = T.tokenize(sentence)
        for tok in tokens:
            if not hobj.spell(tok):
                unknown_words[emotion] += [tok]
                unknown_words_len[emotion] += 1   
        avg_lengths[emotion] += len(tokens)/sums[emotion]
        
for k,v in exclamatives.items():
    exclamatives[k] = exclamatives[k]
for k,v in interrogatives.items():
    interrogatives[k] = interrogatives[k]
for k,v in upper_cases.items():
    upper_cases[k] = upper_cases[k]
for k,v in upper_cases.items():
    lower_cases[k] = lower_cases[k]
for k,v in upper_cases.items():
    emojis_s[k] = emojis_s[k]
for k,v in unknown_words_len.items():
    unknown_words_len[k] = unknown_words_len[k]

    
print('exclamatives: \t\t', exclamatives)
print('interrogatives: \t', interrogatives)
print('upper_cases: \t\t', upper_cases)    
print('lower_cases: \t\t', lower_cases)    
print('emojis: \t\t', emojis_s)    
print('unknown_words_len: \t', unknown_words_len)    
print('avg_lengths: \t\t', avg_lengths)    


exclamatives: 		 {'others': 4449, 'angry': 999, 'sad': 526, 'happy': 435}
interrogatives: 	 {'others': 375, 'angry': 302, 'sad': 142, 'happy': 299}
upper_cases: 		 {'others': 30431, 'angry': 12940, 'sad': 10749, 'happy': 8173}
lower_cases: 		 {'others': 383173, 'angry': 175842, 'sad': 150413, 'happy': 105035}
emojis: 		 {'others': 1100, 'angry': 727, 'sad': 2669, 'happy': 3724}
unknown_words_len: 	 {'others': 15026, 'angry': 5292, 'sad': 6429, 'happy': 7562}
avg_lengths: 		 {'others': 8.1667781643031, 'angry': 10.163458045768357, 'sad': 9.115687351272197, 'happy': 8.477728022625381}
